In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py


from sklearn.preprocessing import MinMaxScaler

In [9]:
PATH = ""

champions = pd.read_csv(PATH + "wc_champions.csv", encoding='utf-8-sig')
players_main = pd.read_csv(PATH + "wc_players_main.csv", encoding='utf-8-sig')
players_pi = pd.read_csv(PATH + "wc_players_play_in.csv", encoding='utf-8-sig')
teams_main = pd.read_csv(PATH + "wc_teams_main.csv", encoding='utf-8-sig')
teams_pi = pd.read_csv(PATH + "wc_teams_play_in.csv", encoding='utf-8-sig')
lol2022season = pd.read_csv(PATH + "2022matchdata.csv" , encoding='utf-8-sig')
lck2022season = pd.read_csv(PATH + "lck2022.csv" , encoding='utf-8-sig')

lol2022season['date'] = pd.to_datetime(lol2022season['date'], format='%Y-%m-%d %H:%M:%S', errors='raise') #날짜데이터전처리

C:\Users\Admin\AppData\Local\Temp\ipykernel_6328\2299288423.py:8: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  lol2022season = pd.read_csv(PATH + "2022matchdata.csv" , encoding='utf-8-sig')


# dataload

In [26]:
lck = lol2022season[ lol2022season['league'] == 'LCK' ]
lec = lol2022season[ lol2022season['league'] == 'LEC' ]
lcs = lol2022season[ lol2022season['league'] == 'LCS' ]
#lpl null값 많아 제외
third_leagues = pd.concat([lck,lec,lcs], axis=0)


lck_spring = lck[ (lck['split'] == 'Spring') & (lck['playoffs']==0) ]
lck_summer = lck[ (lck['split'] == 'Summer') & (lck['playoffs']==0) ]
lck_spring_po = lck[ (lck['split'] == 'Spring') & (lck['playoffs']==1) ]
lck_summer_po = lck[ (lck['split'] == 'Summer') & (lck['playoffs']==1) ]

In [27]:
# 3대리그 중요요인으로 생각하는 피쳐들 상관관계 분석
third_leagues_hm_df = third_leagues[ ['gameid','participantid','side','league','split','teamname','date', \
    'gamelength','result','totalgold','golddiffat15','xpdiffat15','towers'] ]

third_leagues_hm_df_team = third_leagues_hm_df[ (third_leagues_hm_df['participantid'] == 100) | (third_leagues_hm_df['participantid'] == 200 ) ]

third_leagues_hm_df_team.head()

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,xpdiffat15,towers
1450,ESPORTSTMNT01_2700815,100,Blue,LCK,Spring,DRX,2022-01-12 06:20:03,2195,0,63747,4757.0,1540.0,8.0
1451,ESPORTSTMNT01_2700815,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 06:20:03,2195,1,67669,-4757.0,-1540.0,9.0
1462,ESPORTSTMNT01_2690695,100,Blue,LCK,Spring,DRX,2022-01-12 09:02:13,2070,0,60674,-1045.0,-735.0,3.0
1463,ESPORTSTMNT01_2690695,200,Red,LCK,Spring,Liiv SANDBOX,2022-01-12 09:02:13,2070,1,67152,1045.0,735.0,9.0
1486,ESPORTSTMNT01_2690705,100,Blue,LCK,Spring,T1,2022-01-12 10:07:10,2233,1,66455,1309.0,-170.0,7.0


In [35]:
t_l_hm_m = third_leagues_hm_df_team[ ['teamname','gamelength','result','totalgold','golddiffat15','towers'] ].groupby('teamname').mean().round(3)
t_l_hm_m

,gamelength,result,totalgold,golddiffat15,towers
teamname,,,,,
100 Thieves,1951.500,0.600,58600.729,238.257,6.414
Astralis,2076.056,0.278,59932.194,-859.444,4.889
Cloud9,1973.181,0.611,60595.153,483.278,6.583
Counter Logic Gaming,2014.121,0.466,59281.552,-746.776,5.845
DRX,2074.211,0.495,61042.789,-149.495,6.095
DWG KIA,2078.347,0.604,61938.802,693.436,7.010
Dignitas,2017.022,0.326,57811.609,-618.522,4.696
Evil Geniuses,2008.651,0.663,61156.221,527.186,7.233
Excel Esports,2045.383,0.489,60208.915,-285.447,6.553


In [37]:
t_l_hm_m.corr().round(3)

,gamelength,result,totalgold,golddiffat15,towers
gamelength,1.000,-0.291,0.381,-0.169,-0.200
result,-0.291,1.000,0.733,0.793,0.966
totalgold,0.381,0.733,1.000,0.687,0.799
golddiffat15,-0.169,0.793,0.687,1.000,0.825
towers,-0.200,0.966,0.799,0.825,1.000


In [33]:
third_leagues_hm_df_team[ ['gamelength','result','totalgold','golddiffat15','towers','xpdiffat15'] ]

,gamelength,result,totalgold,golddiffat15,towers,xpdiffat15
1450,2195,0,63747,4757.0,8.0,1540.0
1451,2195,1,67669,-4757.0,9.0,-1540.0
1462,2070,0,60674,-1045.0,3.0,-735.0
1463,2070,1,67152,1045.0,9.0,735.0
1486,2233,1,66455,1309.0,7.0,-170.0
...,...,...,...,...,...,...
133319,1661,0,43013,-1549.0,2.0,-2604.0
133354,1493,0,38560,3003.0,2.0,917.0
133355,1493,1,50003,-3003.0,8.0,-917.0
133366,1800,0,46204,195.0,1.0,-1700.0


In [56]:
fig = px.imshow( t_l_hm_m.corr().round(3) , text_auto=True, title="승패와 주요 경기지표와의 상관관계 히트맵" ,width = 500 )

fig.update_layout(
            title_x = 0.5,
            title_y = 0.9,
            title_xanchor = "auto",
            title_yanchor = "auto",
            )

fig.show()

In [54]:
import chart_studio.tools

username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig, filename = 'heatmap1', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/3/'

---
lck 강 중 약 팀 선택

In [483]:
lck_summer_df = lck_summer[ ['gameid','participantid','side','league','split','teamname','date', \
    'gamelength','result','totalgold','golddiffat15','towers'] ]

lck_summer_team_df = lck_summer_df[ (lck_summer_df['participantid'] == 100) | (lck_summer_df['participantid'] == 200 ) ]

lck_summer_team_df['result'] = np.where( lck_summer_team_df['result'] == 0 , 'lose', 'win')

lck_summer_team_df.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6328\560810934.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
78262,ESPORTSTMNT02_2810546,100,Blue,LCK,Summer,Kwangdong Freecs,2022-06-15 06:51:56,2280,win,70874,-647.0,10.0
78263,ESPORTSTMNT02_2810546,200,Red,LCK,Summer,Hanwha Life Esports,2022-06-15 06:51:56,2280,lose,61008,647.0,3.0
78274,ESPORTSTMNT02_2820281,100,Blue,LCK,Summer,Hanwha Life Esports,2022-06-15 09:00:17,2326,win,75752,-1026.0,11.0
78275,ESPORTSTMNT02_2820281,200,Red,LCK,Summer,Kwangdong Freecs,2022-06-15 09:00:17,2326,lose,65096,1026.0,4.0
78298,ESPORTSTMNT02_2820291,100,Blue,LCK,Summer,Kwangdong Freecs,2022-06-15 10:02:25,1906,win,65360,1671.0,9.0


In [484]:
lck_summer_team_df.isna().sum() #결측값확인

gameid           0
participantid    0
side             0
league           0
split            0
teamname         0
date             0
gamelength       0
result           0
totalgold        0
golddiffat15     0
towers           0
dtype: int64

In [485]:
fig = px.box( y= lck_summer_team_df['totalgold'], width=400, height=500 )
# 게임시간이 매우 긴 경기 totalgold 비약적으로 증가
# 따라서 15diff 지표로 확인하는것이 적절
fig.show()

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig, filename = 'heatmap1', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/3/'

In [136]:
Q1 = lck_summer_team_df['totalgold'].quantile(0.25)
Q3 = lck_summer_team_df['totalgold'].quantile(0.75)

IQR = Q3-Q1
out_min = Q1 - (IQR*1.5)
out_max = Q3 + (IQR*1.5)

In [160]:
lck_summer_tg_df = lck_summer_team_df.copy()

lck_summer_tg_df['teamname'] = np.where( (lck_summer_tg_df['totalgold'] >= out_max) , 'outlier',\
     np.where(lck_summer_tg_df['totalgold'] <= out_min , 'outlier', lck_summer_tg_df['teamname'] )
)

In [161]:
lck_summer_tg_df[ lck_summer_tg_df['teamname'] == 'outlier' ]

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
78322,ESPORTSTMNT02_2820303,100,Blue,LCK,Summer,outlier,2022-06-15 10:59:29,2804,lose,88598,-1197.0,6.0
78323,ESPORTSTMNT02_2820303,200,Red,LCK,Summer,outlier,2022-06-15 10:59:29,2804,win,92246,1197.0,11.0
80051,ESPORTSTMNT05_2780070,200,Red,LCK,Summer,outlier,2022-06-17 11:56:24,2788,win,89598,570.0,9.0
99839,ESPORTSTMNT05_2951551,200,Red,LCK,Summer,outlier,2022-07-13 11:25:55,1175,lose,26879,-8521.0,0.0
108562,ESPORTSTMNT05_2990683,100,Blue,LCK,Summer,outlier,2022-07-24 09:09:13,3036,win,88132,-216.0,11.0


In [162]:
lck_summer_outtg_df = lck_summer_tg_df.groupby('teamname')['totalgold'].mean()

lck_summer_outltime_df = lck_summer_tg_df.groupby('teamname')['gamelength'].mean()

In [165]:
lck_summer_outltime_df

teamname
DRX                    2035.977273
DWG KIA                1990.048780
Fredit BRION           2012.261905
Gen.G                  1860.050000
Hanwha Life Esports    1929.391304
KT Rolster             2040.533333
Kwangdong Freecs       2083.261905
Liiv SANDBOX           1858.615385
Nongshim RedForce      1979.414634
T1                     1911.348837
outlier                2521.400000
Name: gamelength, dtype: float64

In [212]:
fig = go.Figure()

colors = ['#5F76C1',] * len(lck_summer_outtg_df.index)
colors[-1] = '#C15F5F'


fig.add_trace(go.Bar(
    x=lck_summer_outtg_df.index , 
    y=lck_summer_outtg_df.values,  
    marker_color=colors,
    text=lck_summer_outtg_df.values, 
    textposition='auto', 
    texttemplate='%{text:.0f} G'
            ) )

fig.update_yaxes(range=[51000, 78000])

fig.update_layout(
            width = 800,
            height = 600,
            title_text="LCK Summer 이상치 팀의 경기 총 골드획득",
            title_x = 0.5,
            title_y = 0.9,
            title_xanchor = "auto",
            title_yanchor = "auto",
            )


fig.show()

In [182]:
lck_summer_outltime_df

teamname
DRX                    2035.977273
DWG KIA                1990.048780
Fredit BRION           2012.261905
Gen.G                  1860.050000
Hanwha Life Esports    1929.391304
KT Rolster             2040.533333
Kwangdong Freecs       2083.261905
Liiv SANDBOX           1858.615385
Nongshim RedForce      1979.414634
T1                     1911.348837
outlier                2521.400000
Name: gamelength, dtype: float64

In [213]:
fig = go.Figure()
colors = ['#5F76C1',] * len(lck_summer_outltime_df.index)
colors[-1] = '#C15F5F'


fig.add_trace(go.Bar(
    x=lck_summer_outltime_df.index , 
    y=lck_summer_outltime_df.values,  
    marker_color=colors,
    text=lck_summer_outltime_df.values, 
    textposition='auto', 
    texttemplate='%{text:.0f}'
            ) )
fig.update_yaxes(range=[1800,2550]) #outlier 강조하기
fig.update_layout(
            width = 800,
            height = 600,
            title_text="LCK Summer 이상치 팀의 경기시간",
            title_x = 0.5,
            title_y = 0.9,
            title_xanchor = "auto",
            title_yanchor = "auto",
            )

fig.show()

In [197]:
# 15분 gold차이 평균 지표
lck_summer_team_golddiff_mean = lck_summer_team_df.groupby('teamname')['golddiffat15'].mean()
lck_summer_team_golddiff_mean

teamname
DRX                    -276.133333
DWG KIA                 655.024390
Fredit BRION           -543.904762
Gen.G                  2008.775000
Hanwha Life Esports   -1190.782609
KT Rolster              249.155556
Kwangdong Freecs       -704.119048
Liiv SANDBOX           -718.390244
Nongshim RedForce      -299.190476
T1                      982.272727
Name: golddiffat15, dtype: float64

In [244]:
fig = go.Figure()
colors = ['#5F76C1',] * len(lck_summer_team_golddiff_mean.index)
colors[1] = '#C15F5F'
colors[3] = '#C15F5F'
colors[5] = '#C15F5F'
colors[9] = '#C15F5F'


fig.add_trace(go.Bar(
    x=lck_summer_team_golddiff_mean.index , 
    y=lck_summer_team_golddiff_mean.values,  
    marker_color=colors,
    text=lck_summer_team_golddiff_mean.values, 
    textposition='auto', 
    texttemplate='%{text:.0f}',
            ) )

fig.update_xaxes(title_text='팀명')
fig.update_yaxes(title_text='15분 당 gold차이')

fig.update_layout(
            width = 900,
            height = 600,
            title_text="LCK Summer 팀 별 15분 상대골드차이",
            title_x = 0.5,
            title_y = 0.9,
            title_xanchor = "auto",
            title_yanchor = "auto",
            )

# fig.add_annotation(
#             x= 400, y= y 좌표,
#             text= 주석 텍스트,
#             textangle= 텍스트 각도,
#             font_color = 텍스트 색,
#             font_family = 텍스트 서체,
#             font_size = 텍스트 사이즈,
#             arrowhead = 화살표 스타일,
#             arrowcolor= 화살표 색,
#             arrowside= 화살표 방향,
#             arrowsize= 화살표 크기,
#             arrowwidth = 화살표 두깨,
#             bgcolor=텍스트 백그라운드색,
#             bordercolor= 테두리 색,
#             borderwidth = 테두리 두깨,
#             opacity = 투명도,
#             xshift = x축 방향 평행이동,
#             yshift = y축 방향 평행이동)
fig.add_annotation(
            x=3,
            y=2000,
            text="<b>2022 summer 우승팀 GenG</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77BDD9",
            ax=40,
            ay=-30,
            bordercolor="#77BDD9",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F22E62",
            opacity=0.8
)

fig.add_annotation(
            x=0,
            y=0,
            text="<b> DRX</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77BDD9",
            ax=10,
            ay=-30,
            bordercolor="#77BDD9",
            borderwidth=2,
            borderpad=4,
            bgcolor="#05E9DA",
            opacity=0.8
)

fig.show()

In [245]:
lck_summer_team_df

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
78262,ESPORTSTMNT02_2810546,100,Blue,LCK,Summer,Kwangdong Freecs,2022-06-15 06:51:56,2280,win,70874,-647.0,10.0
78263,ESPORTSTMNT02_2810546,200,Red,LCK,Summer,Hanwha Life Esports,2022-06-15 06:51:56,2280,lose,61008,647.0,3.0
78274,ESPORTSTMNT02_2820281,100,Blue,LCK,Summer,Hanwha Life Esports,2022-06-15 09:00:17,2326,win,75752,-1026.0,11.0
78275,ESPORTSTMNT02_2820281,200,Red,LCK,Summer,Kwangdong Freecs,2022-06-15 09:00:17,2326,lose,65096,1026.0,4.0
78298,ESPORTSTMNT02_2820291,100,Blue,LCK,Summer,Kwangdong Freecs,2022-06-15 10:02:25,1906,win,65360,1671.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
123323,ESPORTSTMNT04_2471386,200,Red,LCK,Summer,Hanwha Life Esports,2022-08-14 07:38:12,1841,win,65879,6209.0,11.0
123346,ESPORTSTMNT04_2471388,100,Blue,LCK,Summer,Nongshim RedForce,2022-08-14 08:32:13,1970,win,56892,147.0,7.0
123347,ESPORTSTMNT04_2471388,200,Red,LCK,Summer,Kwangdong Freecs,2022-08-14 08:32:13,1970,lose,48813,-147.0,0.0
123430,ESPORTSTMNT04_2471406,100,Blue,LCK,Summer,Kwangdong Freecs,2022-08-14 10:13:38,1471,lose,37185,-3600.0,1.0


In [256]:
lck_summer_golddiff_winrate = lck_summer_team_df[lck_summer_team_df['golddiffat15']>=1000].groupby('result').count()['gameid']

In [281]:
import plotly.express as px

fig = px.pie(values=lck_summer_golddiff_winrate.values, names=lck_summer_golddiff_winrate.index, width=500)

fig.update_layout(
                title_text="golddiff 가 1000이상 팀의 승률",
                title_x = 0.5,
                title_y = 0.96,
                title_xanchor = "auto",
                title_yanchor = "auto")

fig.update_traces(
                hole=.3,
                textinfo = 'label+percent+value',
                textfont_size = 20,
                textfont_color = 'black',
                textposition = 'inside')

fig.show()


lck_summer_golddiff_winrate

result
lose     30
win     112
Name: gameid, dtype: int64

In [284]:
lck_summer_team_df

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
78262,ESPORTSTMNT02_2810546,100,Blue,LCK,Summer,Kwangdong Freecs,2022-06-15 06:51:56,2280,win,70874,-647.0,10.0
78263,ESPORTSTMNT02_2810546,200,Red,LCK,Summer,Hanwha Life Esports,2022-06-15 06:51:56,2280,lose,61008,647.0,3.0
78274,ESPORTSTMNT02_2820281,100,Blue,LCK,Summer,Hanwha Life Esports,2022-06-15 09:00:17,2326,win,75752,-1026.0,11.0
78275,ESPORTSTMNT02_2820281,200,Red,LCK,Summer,Kwangdong Freecs,2022-06-15 09:00:17,2326,lose,65096,1026.0,4.0
78298,ESPORTSTMNT02_2820291,100,Blue,LCK,Summer,Kwangdong Freecs,2022-06-15 10:02:25,1906,win,65360,1671.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...
123323,ESPORTSTMNT04_2471386,200,Red,LCK,Summer,Hanwha Life Esports,2022-08-14 07:38:12,1841,win,65879,6209.0,11.0
123346,ESPORTSTMNT04_2471388,100,Blue,LCK,Summer,Nongshim RedForce,2022-08-14 08:32:13,1970,win,56892,147.0,7.0
123347,ESPORTSTMNT04_2471388,200,Red,LCK,Summer,Kwangdong Freecs,2022-08-14 08:32:13,1970,lose,48813,-147.0,0.0
123430,ESPORTSTMNT04_2471406,100,Blue,LCK,Summer,Kwangdong Freecs,2022-08-14 10:13:38,1471,lose,37185,-3600.0,1.0


In [356]:
lck_summer_drx_df = lck_summer_team_df[lck_summer_team_df['teamname']=='DRX']

lck_summer_geng_df = lck_summer_team_df[lck_summer_team_df['teamname']=='Gen.G']

lck_summer_t1_df = lck_summer_team_df[lck_summer_team_df['teamname']=='T1']

lck_summer_dk_df = lck_summer_team_df[lck_summer_team_df['teamname']=='DWG KIA']

lck_summer_dk_df.head()




,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
79223,ESPORTSTMNT02_2840060,200,Red,LCK,Summer,DWG KIA,2022-06-16 10:27:35,2136,win,60576,1369.0,7.0
79247,ESPORTSTMNT02_2830073,200,Red,LCK,Summer,DWG KIA,2022-06-16 11:48:41,2490,win,77300,5322.0,9.0
81238,ESPORTSTMNT02_2840747,100,Blue,LCK,Summer,DWG KIA,2022-06-19 12:18:26,1681,win,58805,2110.0,9.0
81275,ESPORTSTMNT02_2830829,200,Red,LCK,Summer,DWG KIA,2022-06-19 12:52:05,1645,win,52616,1133.0,7.0
83279,ESPORTSTMNT03_2793981,200,Red,LCK,Summer,DWG KIA,2022-06-22 06:21:27,1801,lose,50380,-341.0,3.0


In [357]:
lck_summer_drx_time_df = lck_summer_drx_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_drx_time_df = lck_summer_drx_time_df.dropna()

lck_summer_geng_time_df = lck_summer_geng_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_geng_time_df = lck_summer_geng_time_df.dropna()


lck_summer_t1_time_df = lck_summer_t1_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_t1_time_df = lck_summer_t1_time_df.dropna()


lck_summer_dk_time_df = lck_summer_dk_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_dk_time_df = lck_summer_dk_time_df.dropna()
lck_summer_dk_time_df.head()

,participantid,gamelength,totalgold,golddiffat15,towers
date,,,,,
2022-06-16,200.000000,2313.0,68938.0,3345.5,8.000000
2022-06-19,150.000000,1663.0,55710.5,1621.5,8.000000
2022-06-22,166.666667,1844.0,53783.0,-1279.0,5.666667
2022-06-25,100.000000,1901.0,53203.5,586.5,2.500000
2022-07-01,150.000000,1965.0,65667.0,9.5,9.500000


In [ ]:
lck_summer_drx_time_df = lck_summer_drx_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_drx_time_df = lck_summer_drx_time_df.dropna()

lck_summer_geng_time_df = lck_summer_geng_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_geng_time_df = lck_summer_geng_time_df.dropna()


lck_summer_t1_time_df = lck_summer_t1_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_t1_time_df = lck_summer_t1_time_df.dropna()


lck_summer_dk_time_df = lck_summer_dk_df.groupby(pd.Grouper(key='date',freq='D')).mean('golddiffat15')
lck_summer_dk_time_df = lck_summer_dk_time_df.dropna()
lck_summer_dk_time_df.head()

,participantid,gamelength,totalgold,golddiffat15,towers
date,,,,,
2022-06-16,200.000000,2313.0,68938.0,3345.5,8.000000
2022-06-19,150.000000,1663.0,55710.5,1621.5,8.000000
2022-06-22,166.666667,1844.0,53783.0,-1279.0,5.666667
2022-06-25,100.000000,1901.0,53203.5,586.5,2.500000
2022-07-01,150.000000,1965.0,65667.0,9.5,9.500000


In [475]:
fig_time = go.Figure()


# 추가 할 그래프 그리기

fig_time.add_trace(go.Scatter(x=lck_summer_drx_time_df.index, y=lck_summer_drx_time_df['golddiffat15'],
                    mode='lines+markers',
                    name='DRX'))

fig_time.add_trace(go.Scatter(x=lck_summer_geng_time_df.index, y=lck_summer_geng_time_df['golddiffat15'],
                    mode='lines',
                    name='Gen.G'))

fig_time.add_trace(go.Scatter(x=lck_summer_t1_time_df.index, y=lck_summer_t1_time_df['golddiffat15'],
                    mode='lines',
                    name='T1'))

fig_time.add_trace(go.Scatter(x=lck_summer_dk_time_df.index, y=lck_summer_dk_time_df['golddiffat15'],
                    mode='lines',
                    name='DWG KIA'))

# fig.add_trace(go.Scatter(x=random_x, y=random_y1,
#                     mode='lines+markers',
#                     name='lines+markers'))
# fig.add_trace(go.Scatter(x=random_x, y=random_y2,
#                     mode='markers', 
#                     name='markers'))

fig_time.update_layout(
                title_text="LCK Summer 경기일자별 golddiff 추세",
                title_x = 0.5,
                title_y = 0.9,
                title_xanchor = "auto",
                title_yanchor = "auto")
fig_time.show()

In [476]:
username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig_time, filename = '', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/7/'

In [368]:
lck_summer_po_df = lck_summer_po[ ['gameid','participantid','side','league','split','teamname','date', \
    'gamelength','result','totalgold','golddiffat15','towers'] ]

lck_summer_po_team_df = lck_summer_po_df[ (lck_summer_po_df['participantid'] == 100) | (lck_summer_po_df['participantid'] == 200 ) ]

lck_summer_po_team_df['result'] = np.where( lck_summer_po_team_df['result'] == 0 , 'lose', 'win')

lck_summer_po_team_df.head(6)

C:\Users\Admin\AppData\Local\Temp\ipykernel_6328\1378851781.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
124666,ESPORTSTMNT04_2451564,100,Blue,LCK,Summer,Liiv SANDBOX,2022-08-17 06:17:37,1832,win,55223,761.0,9.0
124667,ESPORTSTMNT04_2451564,200,Red,LCK,Summer,DRX,2022-08-17 06:17:37,1832,lose,48921,-761.0,3.0
124702,ESPORTSTMNT04_2451579,100,Blue,LCK,Summer,DRX,2022-08-17 08:39:14,1918,lose,51592,1099.0,2.0
124703,ESPORTSTMNT04_2451579,200,Red,LCK,Summer,Liiv SANDBOX,2022-08-17 08:39:14,1918,win,62747,-1099.0,9.0
124738,ESPORTSTMNT04_2451580,100,Blue,LCK,Summer,DRX,2022-08-17 09:33:42,2110,win,67482,342.0,11.0
124739,ESPORTSTMNT04_2451580,200,Red,LCK,Summer,Liiv SANDBOX,2022-08-17 09:33:42,2110,lose,55355,-342.0,1.0


In [367]:
lck_summer_po_team_df[lck_summer_po_team_df['teamname']=='DRX']

,gameid,participantid,side,league,split,teamname,date,gamelength,result,totalgold,golddiffat15,towers
124667,ESPORTSTMNT04_2451564,200,Red,LCK,Summer,DRX,2022-08-17 06:17:37,1832,lose,48921,-761.0,3.0
124702,ESPORTSTMNT04_2451579,100,Blue,LCK,Summer,DRX,2022-08-17 08:39:14,1918,lose,51592,1099.0,2.0
124738,ESPORTSTMNT04_2451580,100,Blue,LCK,Summer,DRX,2022-08-17 09:33:42,2110,win,67482,342.0,11.0
124775,ESPORTSTMNT04_2451582,200,Red,LCK,Summer,DRX,2022-08-17 10:32:27,3074,lose,84181,1205.0,5.0


---

In [395]:
lck_summer_df = lck_summer[ ['gameid','playername','participantid','side','league','split','teamname','date', \
    'totalgold','gamelength','result','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

lck_summer_drxp_df = lck_summer_df[ (lck_summer_df['teamname'] == 'DRX') & (lck_summer_df['participantid'] <= 10 ) ]

lck_summer_gengp_df = lck_summer_df[ (lck_summer_df['teamname'] == 'Gen.G') & (lck_summer_df['participantid'] <= 10 ) ]

lck_summer_t1p_df = lck_summer_df[ (lck_summer_df['teamname'] == 'T1') & (lck_summer_df['participantid'] <= 10 ) ]

lck_summer_dkp_df = lck_summer_df[ (lck_summer_df['teamname'] == 'DWG KIA') & (lck_summer_df['participantid'] <= 10 ) ]

lck_summer_drxp_df.head()

,gameid,playername,participantid,side,league,split,teamname,date,totalgold,gamelength,result,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
78317,ESPORTSTMNT02_2820303,Kingen,6,Red,LCK,Summer,DRX,2022-06-15 10:59:29,23011,2804,1,445.9558,502.1469,6445.0,7860.0,10.2710,82.0
78318,ESPORTSTMNT02_2820303,Pyosik,7,Red,LCK,Summer,DRX,2022-06-15 10:59:29,17328,2804,1,212.6106,732.7960,5587.0,5857.0,5.6491,96.0
78319,ESPORTSTMNT02_2820303,ZEKA,8,Red,LCK,Summer,DRX,2022-06-15 10:59:29,19849,2804,1,415.1213,674.5720,5378.0,7260.0,9.8003,59.0
78320,ESPORTSTMNT02_2820303,Deft,9,Red,LCK,Summer,DRX,2022-06-15 10:59:29,21633,2804,1,1023.7019,573.9800,5357.0,5778.0,9.5007,59.0
78321,ESPORTSTMNT02_2820303,BeryL,10,Red,LCK,Summer,DRX,2022-06-15 10:59:29,10425,2804,1,58.3738,407.7175,3568.0,4175.0,0.8987,146.0


In [396]:
lck_summer_drxp_score = lck_summer_drxp_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

lck_summer_gengp_score = lck_summer_gengp_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

lck_summer_t1p_score = lck_summer_t1p_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]

lck_summer_dkp_score = lck_summer_dkp_df[ ['playername','teamname','dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore'] ]


lck_summer_drxp_score.head()

,playername,teamname,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
78317,Kingen,DRX,445.9558,502.1469,6445.0,7860.0,10.2710,82.0
78318,Pyosik,DRX,212.6106,732.7960,5587.0,5857.0,5.6491,96.0
78319,ZEKA,DRX,415.1213,674.5720,5378.0,7260.0,9.8003,59.0
78320,Deft,DRX,1023.7019,573.9800,5357.0,5778.0,9.5007,59.0
78321,BeryL,DRX,58.3738,407.7175,3568.0,4175.0,0.8987,146.0


In [397]:
X = lck_summer_drxp_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]
X2 = lck_summer_gengp_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]
X3 = lck_summer_t1p_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]
X4 = lck_summer_dkp_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']]

In [452]:
scaler = MinMaxScaler(feature_range=(0,1)) ## 각 칼럼 데이터 값을 0~1 범위로 변환
scaler.fit(X)
scaler.fit(X2)
scaler.fit(X3)
scaler.fit(X4)
 ## 각 칼럼 데이터마다 변환할 함수 생성
transformed_X = scaler.transform(X)
transformed_X2 = scaler.transform(X2)
transformed_X3 = scaler.transform(X3)
transformed_X4 = scaler.transform(X4)


 ## fit에서 만들어진 함수를 실제로 데이터에 적용
transformed_X

array([[0.31387567, 0.36105248, 0.7106599 , 0.90336058, 0.68927431,
        0.50769231],
       [0.12587431, 0.56466945, 0.5364467 , 0.53950954, 0.37454461,
        0.61538462],
       [0.28903296, 0.51326932, 0.49401015, 0.79436876, 0.65722185,
        0.33076923],
       ...,
       [0.26993546, 0.290317  , 0.47106599, 0.83324251, 0.59573179,
        0.06923077],
       [0.29908171, 0.27929763, 0.42010152, 0.39836512, 0.62902358,
        0.12307692],
       [0.04632341, 0.28646164, 0.06680203, 0.23215259, 0.07420346,
        0.46153846]])

In [453]:
lck_summer_drxp_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X

lck_summer_gengp_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X2

lck_summer_t1p_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X3

lck_summer_dkp_score[['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']] = transformed_X4

lck_summer_t1p_score.head()

C:\Users\Admin\AppData\Local\Temp\ipykernel_6328\2021941436.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\AppData\Local\Temp\ipykernel_6328\2021941436.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Admin\AppData\Local\Temp\ipykernel_6328\2021941436.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,playername,teamname,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
80004,Zeus,T1,0.576440,0.480389,0.602843,0.907720,0.669928,0.200000
80005,Oner,T1,0.261645,0.507555,0.401015,0.461035,0.395048,0.307692
80006,Faker,T1,0.284220,0.363114,0.520812,0.812716,0.627090,0.284615
80007,Gumayusi,T1,0.364873,0.020361,0.160000,0.137693,0.132663,1.253846
80008,Keria,T1,0.142641,0.276847,0.439391,0.551135,0.684208,0.307692


In [454]:
drx_summer_score = lck_summer_drxp_score.groupby('playername')\
    [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()

geng_summer_score = lck_summer_gengp_score.groupby('playername')\
    [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()

t1_summer_score = lck_summer_t1p_score.groupby('playername')\
    [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()

dk_summer_score = lck_summer_dkp_score.groupby('playername')\
    [['dpm','damagetakenperminute','goldat15','xpat15','cspm','visionscore']].mean()




drx_summer_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
BeryL,0.091629,0.322552,0.158312,0.230203,0.120393,0.525812
Deft,0.422516,0.294916,0.495265,0.446620,0.563941,0.345983
Juhan,0.139854,0.547867,0.366159,0.372994,0.481484,0.274359
Kingen,0.354269,0.604925,0.460738,0.836662,0.523728,0.162735
Pyosik,0.168686,0.617106,0.410950,0.486826,0.398187,0.286630
ZEKA,0.425493,0.378441,0.528632,0.828616,0.623830,0.164786


In [455]:
drx_summer_score_df1 = pd.DataFrame( { "direction" : drx_summer_score.columns ,
"player" : [drx_summer_score.index[0]] * 6 ,
"stats" : drx_summer_score.iloc[0].values }
)

drx_summer_score_df2 = pd.DataFrame( { "direction" : drx_summer_score.columns ,
"player" : [drx_summer_score.index[1]] * 6 ,
"stats" : drx_summer_score.iloc[1].values }
)

drx_summer_score_df3 = pd.DataFrame( { "direction" : drx_summer_score.columns ,
"player" : [drx_summer_score.index[5]] * 6 ,
"stats" : drx_summer_score.iloc[5].values }
)

drx_summer_score_df4 = pd.DataFrame( { "direction" : drx_summer_score.columns ,
"player" : [drx_summer_score.index[4]] * 6 ,
"stats" : drx_summer_score.iloc[4].values }
)

drx_summer_score_df5 = pd.DataFrame( { "direction" : drx_summer_score.columns ,
"player" : [drx_summer_score.index[3]] * 6 ,
"stats" : drx_summer_score.iloc[3].values }
)


drx_summer_score_df = pd.concat( [drx_summer_score_df1,drx_summer_score_df2,drx_summer_score_df3,drx_summer_score_df4,drx_summer_score_df5]\
    ,axis=0)

drx_summer_score_df

,direction,player,stats
0,dpm,BeryL,0.091629
1,damagetakenperminute,BeryL,0.322552
2,goldat15,BeryL,0.158312
3,xpat15,BeryL,0.230203
4,cspm,BeryL,0.120393
5,visionscore,BeryL,0.525812
0,dpm,Deft,0.422516
1,damagetakenperminute,Deft,0.294916
2,goldat15,Deft,0.495265
3,xpat15,Deft,0.446620


In [456]:
geng_summer_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
Chovy,0.437216,0.331229,0.598909,0.888819,0.675920,0.279231
Doran,0.395149,0.572139,0.538452,0.869401,0.577684,0.113846
Lehends,0.099411,0.232038,0.114640,0.194478,0.078033,0.490962
Peanut,0.165431,0.602828,0.479228,0.535863,0.404298,0.255385
Ruler,0.495687,0.299317,0.603142,0.493960,0.640057,0.318846


In [446]:
geng_summer_score_df1 = pd.DataFrame( { "direction" : geng_summer_score.columns ,
"player" : [geng_summer_score.index[2]] * 6 ,
"stats" : geng_summer_score.iloc[2].values }
)

geng_summer_score_df2 = pd.DataFrame( { "direction" : geng_summer_score.columns ,
"player" : [geng_summer_score.index[4]] * 6 ,
"stats" : geng_summer_score.iloc[4].values }
)

geng_summer_score_df3 = pd.DataFrame( { "direction" : geng_summer_score.columns ,
"player" : [geng_summer_score.index[0]] * 6 ,
"stats" : geng_summer_score.iloc[0].values }
)

geng_summer_score_df4 = pd.DataFrame( { "direction" : geng_summer_score.columns ,
"player" : [geng_summer_score.index[3]] * 6 ,
"stats" : geng_summer_score.iloc[3].values }
)

geng_summer_score_df5 = pd.DataFrame( { "direction" : geng_summer_score.columns ,
"player" : [geng_summer_score.index[1]] * 6 ,
"stats" : geng_summer_score.iloc[1].values }
)


geng_summer_score_df = pd.concat( [geng_summer_score_df1,geng_summer_score_df2,geng_summer_score_df3, geng_summer_score_df4 , geng_summer_score_df5]\
    ,axis=0)

geng_summer_score_df

,direction,player,stats
0,dpm,Lehends,0.099411
1,damagetakenperminute,Lehends,0.232038
2,goldat15,Lehends,0.114640
3,xpat15,Lehends,0.194478
4,cspm,Lehends,0.078033
5,visionscore,Lehends,0.490962
0,dpm,Ruler,0.495687
1,damagetakenperminute,Ruler,0.299317
2,goldat15,Ruler,0.603142
3,xpat15,Ruler,0.493960


In [478]:
import plotly.express as px

fig1 = px.line_polar(drx_summer_score_df , r='stats' , theta='direction', color='player', line_close=True, range_r=(0,1) ,width = 500) 
fig2 = px.line_polar(geng_summer_score_df, r='stats', theta='direction', color='player', line_close=True, range_r=(0,1) ,width = 500)
fig3 = px.line_polar(t1_summer_score_df, r='stats', theta='direction', color='player', line_close=True, range_r=(0,1) ,width = 500)
fig4 = px.line_polar(dk_summer_score_df, r='stats', theta='direction', color='player', line_close=True, range_r=(0,1) ,width = 500)

fig1.show()

In [479]:
import chart_studio.tools

username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig1, filename = 'radermap1', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/9/'

In [467]:
fig2.show()

In [480]:
username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig2, filename = 'radermap1', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/9/'

In [468]:
fig3.show()

In [481]:
username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig3, filename = 'radermap3', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/12/'

In [469]:
fig4.show()

In [482]:
username = 'damidaa'  # 자신의 username (plotly account를 만들어야 함)
api_key = 'OuvIp3oI14tS4Sthfgrv'    # 자신의 api key (settings > regenerate key)
  
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

py.plot(fig4, filename = 'radermap4', auto_open=False)  # fig: 작성한 차트를 저장한 변수
  ## 위 코드를 실행하면 새로운 window로 해당 차트의 링크가 열리고, notebook에도 link를 아래에 return해줌

'https://plotly.com/~damidaa/14/'

In [450]:
t1_summer_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
Faker,0.379869,0.366584,0.518999,0.806362,0.586778,0.183916
Gumayusi,0.369756,0.237055,0.526465,0.428214,0.606933,0.237238
Keria,0.106588,0.228089,0.147097,0.211580,0.110802,0.545105
Oner,0.191085,0.604425,0.462658,0.508389,0.407929,0.200699
Zeus,0.400026,0.588599,0.529622,0.828387,0.544016,0.125000


In [462]:
t1_summer_score_df1 = pd.DataFrame( { "direction" : t1_summer_score.columns ,
"player" : [t1_summer_score.index[2]] * 6 ,
"stats" : t1_summer_score.iloc[2].values }
)

t1_summer_score_df2 = pd.DataFrame( { "direction" : t1_summer_score.columns ,
"player" : [t1_summer_score.index[1]] * 6 ,
"stats" : t1_summer_score.iloc[1].values }
)

t1_summer_score_df3 = pd.DataFrame( { "direction" : t1_summer_score.columns ,
"player" : [t1_summer_score.index[0]] * 6 ,
"stats" : t1_summer_score.iloc[0].values }
)

t1_summer_score_df4 = pd.DataFrame( { "direction" : t1_summer_score.columns ,
"player" : [t1_summer_score.index[3]] * 6 ,
"stats" : t1_summer_score.iloc[3].values }
)

t1_summer_score_df5 = pd.DataFrame( { "direction" : t1_summer_score.columns ,
"player" : [t1_summer_score.index[4]] * 6 ,
"stats" : t1_summer_score.iloc[4].values }
)


t1_summer_score_df = pd.concat( [t1_summer_score_df1 , t1_summer_score_df2 , t1_summer_score_df3 , t1_summer_score_df4 , t1_summer_score_df5]\
    ,axis=0)

t1_summer_score_df.head()

,direction,player,stats
0,dpm,Keria,0.106588
1,damagetakenperminute,Keria,0.228089
2,goldat15,Keria,0.147097
3,xpat15,Keria,0.211580
4,cspm,Keria,0.110802


In [463]:
dk_summer_score

,dpm,damagetakenperminute,goldat15,xpat15,cspm,visionscore
playername,,,,,,
Bible,0.059637,0.344219,0.113015,0.186994,0.050577,0.466154
Canyon,0.228697,0.633506,0.431150,0.512146,0.419654,0.281238
Kellin,0.117288,0.165990,0.122719,0.186109,0.051390,0.551923
Nuguri,0.372019,0.594395,0.531742,0.867819,0.536974,0.164728
ShowMaker,0.356895,0.329190,0.535199,0.867881,0.597292,0.228330
deokdam,0.450958,0.283253,0.542905,0.451678,0.612530,0.191745


In [465]:
dk_summer_score_df1 = pd.DataFrame( { "direction" : dk_summer_score.columns ,
"player" : [dk_summer_score.index[2]] * 6 ,
"stats" : dk_summer_score.iloc[2].values }
)

dk_summer_score_df2 = pd.DataFrame( { "direction" : dk_summer_score.columns ,
"player" : [dk_summer_score.index[5]] * 6 ,
"stats" : dk_summer_score.iloc[5].values }
)

dk_summer_score_df3 = pd.DataFrame( { "direction" : dk_summer_score.columns ,
"player" : [dk_summer_score.index[4]] * 6 ,
"stats" : dk_summer_score.iloc[4].values }
)

dk_summer_score_df4 = pd.DataFrame( { "direction" : dk_summer_score.columns ,
"player" : [dk_summer_score.index[1]] * 6 ,
"stats" : dk_summer_score.iloc[1].values }
)

dk_summer_score_df5 = pd.DataFrame( { "direction" : dk_summer_score.columns ,
"player" : [dk_summer_score.index[3]] * 6 ,
"stats" : dk_summer_score.iloc[3].values }
)


dk_summer_score_df = pd.concat( [dk_summer_score_df1 , dk_summer_score_df2 , dk_summer_score_df3 , dk_summer_score_df4 , dk_summer_score_df5]\
    ,axis=0)

dk_summer_score_df.head()

,direction,player,stats
0,dpm,Kellin,0.117288
1,damagetakenperminute,Kellin,0.165990
2,goldat15,Kellin,0.122719
3,xpat15,Kellin,0.186109
4,cspm,Kellin,0.051390
